In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import re

In [2]:
def getSearchPhrases():
    scope = ['https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open("DIGHUM Sentiment")
    sheet = sheet.worksheet("searches")
    df = pd.DataFrame(sheet.get_all_values())
    if not df.empty:
        df.columns = df.iloc[0]
        df = df.iloc[1:]
    return df
searchPhrasesDf = getSearchPhrases()

In [3]:
articlesDf = pd.DataFrame(columns=["query", "date","source", "title", "description"])
for query in searchPhrasesDf ["Phrases"]:
    print(query)
    preppedQuery = re.sub(" ", "%20", query)
    url = "https://news.yahoo.com/rss/search?q=" + preppedQuery + "when:1d"
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib') 
    dates = soup.find_all('pubdate')
    titles = soup.find_all('title')
    descriptions = soup.find_all('description')
    sources = soup.find_all('source')
    print(sources[0])
    print(len(dates))
    print(len(titles))
    for x in range(1, len(descriptions) - 1):
        articlesDf = articlesDf.append({"query" : query,
                                        "date" : dates[x].getText(),
                                        "source": sources[x - 1].next_sibling,
                                        'title': titles[x + 1].getText(), 
                                        "description": descriptions[x].get_text().split('/a>')[1].split('<')[0]}, ignore_index=True)

Black Lives Matter
https://news.yahoo.com/rss/search?q=Black%20Lives%20Matterwhen:1d
<source url="https://news.yahoo.com/"/>
51
52
All Lives Matter
https://news.yahoo.com/rss/search?q=All%20Lives%20Matterwhen:1d
<source url="https://news.yahoo.com/"/>
51
52
Blue Lives Matter
https://news.yahoo.com/rss/search?q=Blue%20Lives%20Matterwhen:1d
<source url="https://news.yahoo.com/"/>
51
52
Donald Trump
https://news.yahoo.com/rss/search?q=Donald%20Trumpwhen:1d
<source url="https://news.yahoo.com/"/>
51
52
Joe Biden
https://news.yahoo.com/rss/search?q=Joe%20Bidenwhen:1d
<source url="https://news.yahoo.com/"/>
51
52
Republican
https://news.yahoo.com/rss/search?q=Republicanwhen:1d
<source url="https://news.yahoo.com/"/>
51
52
Democratic
https://news.yahoo.com/rss/search?q=Democraticwhen:1d
<source url="https://news.yahoo.com/"/>
51
52


In [4]:
scope = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("DIGHUM Sentiment")
sheet = sheet.worksheet("YahooNews")
df = pd.DataFrame(sheet.get_all_values())
if not df.empty:
    df.columns =  df.iloc[0]
    df = df.iloc[1:]
newDfForSheet = articlesDf.append(df)
newindex = [x for x in range(len(newDfForSheet["query"]))]
newDfForSheet["check"] = newindex
newDfForSheet = newDfForSheet.set_index("check")
spreadsheet_key = '1YgFoSOEt9edjE0CTmMayAN6J09zMHdI-WQu9BL758Ek'
wks_name = "YahooNews"
d2g.upload(newDfForSheet, spreadsheet_key, wks_name, credentials=creds, row_names = False)

<Worksheet 'YahooNews' id:325347907>